In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE40914'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression analysis and in silico characterization of intronic long noncoding RNAs in renal cell carcinoma: emerging functional associations"
!Series_summary	"Intronic and intergenic long noncoding RNAs (lncRNAs) are emerging gene expression regulators. The molecular pathogenesis of renal cell carcinoma (RCC) is still poorly understood, and in particular, limited studies are available for intronic lncRNAs expressed in RCC. Microarray experiments were performed with two different custom-designed arrays enriched with probes for lncRNAs mapping to intronic genomic regions. Samples from 18 primary clear cell RCC tumors and 11 nontumor adjacent matched tissues were analyzed with 4k-probes microarrays. Oligoarrays with 44k-probes were used to interrogate 17 RCC samples (14 clear cell, 2 papillary, 1 chromophobe subtypes) split into four pools. Meta-analyses were performed by taking the genomic coordinates of the RCC-expressed lncRNAs, and cross-referenc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = None
gender_row = None
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

def convert_trait(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    if value.lower() == 'renal cell carcinoma (rcc)':
        return 1
    return None

def convert_age(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    if ':' in value:
        value = value.split(':')[1].strip()
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE40914', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['3', '4', '6', '7', '9', '10', '15', '16', '17', '19', '20', '21', '22',
       '23', '26', '27', '31', '32', '33', '35'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['910', '4260', '1981', '2381', '4288'], 'GB_ACC': ['BE833259', 'BE702227', 'BF364095', 'BE081005', 'AW880607'], 'SPOT_TYPE': ['Exonic', 'Exonic', 'Exonic', 'Exonic', 'Exonic'], 'GENE_ID': ['85439', '2776', '84131', '2776', '54768'], 'GENE_SYMBOL': ['STON2', 'GNAQ', 'CEP78', 'GNAQ', 'HYDIN'], 'GENE_ANNOTATION': ['stonin 2', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'centrosomal protein 78kDa', 'Guanine nucleotide binding protein (G protein), q polypeptide', 'hydrocephalus inducing homolog 2 (mouse); hydrocephalus inducing homolog (mouse)'], 'CPC_CODING_POTENTIAL': ['noncoding', 'noncoding', 'noncoding', 'noncoding', '-'], 'SEQUENCE': ['CTGATCCGCTTAAGCTTAGTATGTTTGAGTGTGTAATTTTAGTTTCTTTTCTGGTTGTATTTGTGGTAGTCAGATGTGTTGGATTGATTCCAACTGGACAGAGTAAGGAATTCCAGCATCCTCTTCCTGCTTGCTCGTGTTACCCCACAGATCAAACCCTCAATTCTAGTTGGGGATGCTGTCTAGCCCCACACCATGACTGAAGCCTTAAGCACTGTTGCGCCTCCATGTGCTTTGGATCAGCAACCCCAGTGGTATTCTACCAGAGCATTGTGGGAAAGCAGATGTATAGTCAGGTCCC

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifiers and gene symbols in the dictionary
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Ensure data types are correct before merging
gene_data = gene_data.apply(pd.to_numeric, errors='coerce')

# 3. Apply the mapping to the gene expression dataframe
try:
    gene_data = apply_gene_mapping(gene_data, mapping_df)
except Exception as e:
    print(f"Error during the apply_gene_mapping step: {e}")
    print("Merged DataFrame preview for debugging:")
    temp_gene_expr = mapping_df.join(gene_data, on='ID')
    print(temp_gene_expr.head())


import pandas as pd
import os

# Ensure the dirs exist
os.makedirs('./preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data', exist_ok=True)
os.makedirs('./preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data', exist_ok=True)

# Assuming clinical_data, convert_trait, convert_age, convert_gender, trait_row, age_row, gender_row already exist from STEP 2
selected_clinical_data_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE40914.csv'

if not os.path.exists(selected_clinical_data_path):
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    selected_clinical_data.to_csv(selected_clinical_data_path)
else:
    selected_clinical_data = pd.read_csv(selected_clinical_data_path, index_col=0)

# Ensure gene_data exists from previous steps


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE40914.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# 4. Save the cohort information.
save_cohort_info('GSE40914', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a CSV file.
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE40914.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Kidney_Papillary_Cell_Carcinoma':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.

